In [1]:
import numpy as np
import pandas as pd
import scipy
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader,random_split,sampler
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from PIL import Image
from tqdm.auto import tqdm
import timm
import sklearn
from sklearn.metrics import balanced_accuracy_score
import torchvision
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
device = "cuda" if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [4]:
test_csv = pd.read_csv('/kaggle/input/UBC-OCEAN/test.csv')

In [5]:
transform_tma = A.Compose(
[
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ColorJitter(),
    A.Resize(4000,4000),
    A.Normalize(mean=(0.485,0.456,0.406),std=(0.229,0.224,0.225)),
    ToTensorV2()
]
)
transform_ntma = A.Compose(
[
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ColorJitter(),
    A.Resize(3000,3000),
    A.Normalize(mean=(0.485,0.456,0.406),std=(0.229,0.224,0.225)),
    A.PadIfNeeded(min_height=4000,min_width=4000,border_mode=cv2.BORDER_CONSTANT,value=(0,0,0)),
    ToTensorV2()
]
)

In [6]:
base_model = timm.create_model('resnet101',pretrained=False)
last_layer = nn.Sequential(
    nn.Linear(in_features=base_model.num_features,out_features = 1000),
    nn.BatchNorm1d(1000),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(1000,6)
)
base_model.fc = last_layer
#print(base_model)

In [7]:
tma_layer = nn.Sequential(
    nn.Conv2d(3,32,(7,7),(5,5)),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.Conv2d(32,128,(6,6),(4,4),50),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128,3,(1,1)),
    nn.BatchNorm2d(3),
    nn.ReLU()
)
ntma_layer = nn.Sequential(
    torchvision.transforms.CenterCrop(3000),
    nn.Conv2d(3,32,(7,7),(5,5)),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.Conv2d(32,128,(5,5),(3,3),38),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128,3,(1,1)),
    nn.BatchNorm2d(3),
    nn.ReLU()
)

In [8]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.tma = tma_layer
        self.ntma = ntma_layer
        self.base = base_model
    
    def forward(self,x,img_type):
        tma_indices = img_type==1
        ntma_indices = img_type==0
        tma_inputs = x[tma_indices]
        ntma_inputs = x[ntma_indices]
        tma_outputs = self.tma(tma_inputs) if tma_inputs.size(0)>0 else torch.empty(0).to(device)
        ntma_outputs = self.ntma(ntma_inputs) if ntma_inputs.size(0)>0 else torch.empty(0).to(device)
        emp = torch.cat((tma_outputs,ntma_outputs))
        out = self.base(emp)
        return out

In [9]:
my_model = Model()
my_model.load_state_dict(torch.load("/kaggle/input/anant11-1/model(16).pt"))

<All keys matched successfully>

In [10]:
def test_func(test_loader,model,device):
    model.to(device).eval()
    predictions = np.zeros((0,6))
    for i,data in enumerate(tqdm(test_loader,total = len(test_loader))):
        images = data['image'].to(device)
        img_type = torch.tensor(data['img_type']).to(device)
        with torch.no_grad():
            output = model(images,img_type)
            predictions = np.concatenate((predictions,output.softmax(1).to('cpu').numpy()),axis=0)
    predictions = np.argmax(predictions,axis=1)
    return predictions    

In [11]:
class testDataset(Dataset):
    def __init__(self,transform=(None,None)):
        self.dir1 = "/kaggle/input/UBC-OCEAN/test_images/"
        self.dir2 = "/kaggle/input/UBC-OCEAN/test_thumbnails/"
        self.transform_tma = transform[0]
        self.transform_ntma = transform[1]
    def __len__(self):
        return len(os.listdir(self.dir1))
    def __getitem__(self,idx):
        img_id = test_csv['image_id'][idx]
        h = test_csv['image_height'][idx]
        w = test_csv['image_width'][idx]
        if((h<10000)and(w<10000)):
            img = cv2.imread(self.dir1+str(img_id)+'.png')
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img = self.transform_tma(image=img)
            img_type = 1
        else:
            img = cv2.imread(self.dir2+str(img_id)+'_thumbnail.png')
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img = self.transform_ntma(image=img)
            img_type = 0
        return{
            'image':img['image'],
            'img_type':img_type,
        }

In [12]:
test_dataset = testDataset(transform=(transform_tma,transform_ntma))
test_loader = DataLoader(test_dataset,batch_size=4,shuffle=False)
pred = test_func(test_loader,my_model,device)

  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
print(pred)

[1]


In [14]:
sub = pd.read_csv("/kaggle/input/UBC-OCEAN/test.csv")
sub = sub.drop(["image_width","image_height"],axis=1)
sub["label"] = pred
sub['label'] = sub['label'].map({0:"CC",1:"EC",2:"HGSC",3:"LGSC",4:"MC",5:"Other"})
sub

,image_id,label
0,41,EC


In [15]:
sub.to_csv('submission.csv',index=False)